# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_preprocessed.csv')
test = pd.read_csv('../../data/01_test_preprocessed.csv')
pred = pd.read_csv('../../data/01_pred_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-12-18 12:22:42,Potsdam: Ausstellung befasst sich mit Justizmi...,NaN,News,NaN,Zeit,Potsdam: Ausstellung befasst sich mit Justizmi...
1,2023-07-16 10:51:03,Heizdebakel und AfD-Hoch - Auf Sommerreise bek...,Sommer des Missvergnügens: Zur Halbzeit der Ko...,Deutschland,NaN,Focus,Heizdebakel und AfD-Hoch - Auf Sommerreise bek...
2,2022-09-26 20:36:37,Allianz der Ausgestoßenen - Wie der Iran Putin...,Russland kann viel vom stark sanktionierten Ir...,Ukraine-Krise,NaN,Focus,Allianz der Ausgestoßenen - Wie der Iran Putin...
3,2022-08-04 08:23:25,Im Newsticker - China-Taiwan-Konflikt: China s...,China reagiert wütend auf den Besuch von Nancy...,Ausland,NaN,Focus,Im Newsticker - China-Taiwan-Konflikt: China s...
4,2023-03-08 02:37:24,US-Regierung erwägt Inhaftierung von Migranten...,Bald soll eine kontroverse US-Asylbeschränkung...,Missing,NaN,Tagesschau,US-Regierung erwägt Inhaftierung von Migranten...


In [4]:
test.head()

,date,title,description,category,text,source,title_description_text
0,2022-10-14 14:23:59,Nordkorea feuert Hunderte Geschosse ab,Raketentests und an die 300 Artilleriegeschoss...,Missing,NaN,Tagesschau,Nordkorea feuert Hunderte Geschosse ab - Raket...
1,2023-03-15 14:40:00,Deutsche Emissionen sinken zu langsam,Deutschland hat 2022 weniger Treibhausgase fre...,Other,NaN,ZDF heute,Deutsche Emissionen sinken zu langsam - Deutsc...
2,2022-09-04 04:40:46,Werner Herzog: Ein lebenslang abenteuerliches ...,Seine Biografie ist eine einzige Aneinanderrei...,Other,NaN,Zeit,Werner Herzog: Ein lebenslang abenteuerliches ...
3,2022-07-22 12:31:19,Soziales: Scholz sagt Bürgergeld ab 2023 zu - ...,Das Bürgergeld soll nach Aussage von Bundeskan...,Other,NaN,stern,Soziales: Scholz sagt Bürgergeld ab 2023 zu - ...
4,2023-08-23 10:42:36,Selbstbestimmungsgesetz: Ein Schritt in Richtu...,Lange wurde über das Selbstbestimmungsgesetz g...,Deutschland,NaN,Zeit,Selbstbestimmungsgesetz: Ein Schritt in Richtu...


## Load German stopwords

In [5]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [6]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<54010x21586 sparse matrix of type '<class 'numpy.int64'>'
	with 277413 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

21586

In [14]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'или', 'российские', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

7

In [16]:
lda.components_

array([[ 0.14285724, 45.00769289,  0.14285768, ...,  0.14285778,
         0.14285741,  0.14285769],
       [ 2.14265927, 67.73431258,  0.14285759, ...,  0.14285769,
         0.14285737,  0.14285761],
       [ 0.14285722, 42.82589072,  1.13971317, ...,  0.14285769,
         0.14285738,  0.14285762],
       ...,
       [ 0.14285723, 50.65704713,  0.14285762, ...,  1.14272807,
         0.14285739,  2.14285418],
       [ 0.1430546 , 27.50387518,  0.14568351, ...,  0.14285777,
         0.14285741,  0.14285769],
       [ 0.14285723, 86.02621569,  1.14317286, ...,  1.14298333,
         2.14285567,  0.14285762]])

In [17]:
len(lda.components_[0])

21586

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 4925, 16667, 15886, ..., 11657,  4025, 15841], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

türkei
dax
kanzler
nato
gastbeitrag
inflation
klima
marktbericht
deutschland
scholz


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['verletzt', 'fragen', 'fordert', 'steingart', 'gabor', 'türkei', 'dax', 'kanzler', 'nato', 'gastbeitrag', 'inflation', 'klima', 'marktbericht', 'deutschland', 'scholz']


Die TOP-15 Wörter für das Thema #1
['grüne', 'un', 'fdp', 'streit', 'lindner', 'eu', 'putin', 'tote', 'regierung', 'macht', 'habeck', 'ampel', 'spd', 'analyse', 'neue']


Die TOP-15 Wörter für das Thema #2
['jahre', 'gericht', 'israel', 'chef', 'haft', 'polizei', 'generation', 'letzte', 'biden', 'wegen', 'präsident', 'us', 'berlin', 'trump', 'ex']


Die TOP-15 Wörter für das Thema #3
['russischer', 'entwicklungen', 'kiew', 'putin', 'nachrichten', 'russen', 'stimmen', 'usa', 'invasion', 'us', 'lage', 'russland', 'russische', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #4
['scholz', 'neue', 'baerbock', 'moskau', 'g7', 'usa', 'deutschland', 'selenskyj', 'gipfel', 'china', 'liveblog', 'putin', 'russland', 'ukraine', 'eu']


Die TOP-15 Wörter für das Thema #5
['länder', 'deutsc

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(54010, 7)

In [24]:
topic_results[0].round(5)

array([0.30036, 0.02042, 0.02041, 0.39232, 0.2256 , 0.02043, 0.02046])

In [25]:
topic_results[0].argmax()

3

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,title_description_text,topic
0,2022-12-18 12:22:42,Potsdam: Ausstellung befasst sich mit Justizmi...,NaN,News,NaN,Zeit,Potsdam: Ausstellung befasst sich mit Justizmi...,3
1,2023-07-16 10:51:03,Heizdebakel und AfD-Hoch - Auf Sommerreise bek...,Sommer des Missvergnügens: Zur Halbzeit der Ko...,Deutschland,NaN,Focus,Heizdebakel und AfD-Hoch - Auf Sommerreise bek...,1
2,2022-09-26 20:36:37,Allianz der Ausgestoßenen - Wie der Iran Putin...,Russland kann viel vom stark sanktionierten Ir...,Ukraine-Krise,NaN,Focus,Allianz der Ausgestoßenen - Wie der Iran Putin...,4
3,2022-08-04 08:23:25,Im Newsticker - China-Taiwan-Konflikt: China s...,China reagiert wütend auf den Besuch von Nancy...,Ausland,NaN,Focus,Im Newsticker - China-Taiwan-Konflikt: China s...,4
4,2023-03-08 02:37:24,US-Regierung erwägt Inhaftierung von Migranten...,Bald soll eine kontroverse US-Asylbeschränkung...,Missing,NaN,Tagesschau,US-Regierung erwägt Inhaftierung von Migranten...,3


In [28]:
#Save result to csv
train.to_csv('train_lda_topics_7.csv')